In [1]:
%matplotlib inline

import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from ipywidgets import interact
import matplotlib.patches as patches
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import *
import pandas as pd

import json

In [2]:
root_path = '/opt/ml/input/data/boostcamp/images'
anno_root = '/opt/ml/input/data/boostcamp/annotation.json'

In [3]:
# UFO format
UFO_anns = []

with open(anno_root, 'r') as f:
    train_json = json.load(f)
    images = train_json['images']
    images_df = pd.DataFrame.from_dict(images)

image_id = list(images_df)

fnames = [os.path.join(root_path, i) for i in image_id] # 이미지 정렬 순서 고려하기 위해, json 파일에서 image path 가져오기 

for index, img_id in enumerate(image_id):
    temp_anns = []

    temp = images_df[f'{img_id}']
    words = temp.loc['words']

    for key in words.keys():
        temp_ann = {}
        temp_ann['image_id'] = img_id 
        temp_ann['id'] = index

        word = words[key]

        if word['illegibility'] is True: # 제외 영역
            temp_ann['category_id'] = 2

        
        # category id
        if word['language'] is not None: 
            if 'ko' in word['language'] and 'en' in word['language']: # ['ko', 'en']
                temp_ann['category_id'] = 4
            elif 'ko' in word['language']: # ['ko']
                temp_ann['category_id'] = 0
            elif 'en' in word['language']: # ['en']
                temp_ann['category_id'] = 1
            elif 'others' in word['language']: # ['others']
                temp_ann['category_id'] = 3
            else: # [] # number
                temp_ann['category_id'] = 5 



        temp_ann['bbox'] = word['points']
        temp_anns.append(temp_ann)
    UFO_anns.append(temp_anns)

In [4]:
lang = {0 : 'ko', 1 : 'en', 2: 'Except', 3:'others', 4:'ko and en', 5:'None'}

In [5]:
palette = sns.color_palette('bright')
img_save_dir = '/opt/ml/wrong_img'
os.makedirs(img_save_dir, exist_ok=True)

img_name_list = [img.split("/")[-1][:-4] for img in fnames]

@interact(idx=(0, len(fnames)-1), cls_id=(0, 6))
def showImg(idx=1, cls_id=6):
    fig, ax = plt.subplots(dpi=300)
    img = io.imread(fnames[idx])

    anns = UFO_anns[idx]

    ax.imshow(img)
    for ann in anns:
        ann_id = ann['id']
        class_idx = ann['category_id']

        ax.set_title(f"{img_name_list[idx]}", fontsize=7)

        # 축 제거 
        ax.set_xticks([])
        ax.set_yticks([])
        for pos in ['right', 'top', 'bottom', 'left']:
            ax.spines[pos].set_visible(False)

        points = np.array(ann['bbox'])

        # bbox 시각화 
        if(class_idx == cls_id or cls_id == 6):
            color = palette[class_idx]
            ax.add_patch(
                patches.Polygon(
                    points,
                    closed=True,
                    edgecolor=color,
                    fill=False,
                    ),
                )

            x, y = points[-1]

            text_y = y-15 if y>15 else y+15 
            plt_text = ax.text(x+8,text_y, f'{lang[class_idx]}_{ann_id}', color='white', fontsize='3', weight='semibold', backgroundcolor=color)
            plt_text.set_bbox(dict(
                facecolor=palette[class_idx],  # background color
                alpha=0.6,  # background alpha
                edgecolor='none',  # border color
                pad=2
            ))

interactive(children=(IntSlider(value=1, description='idx', max=1287), IntSlider(value=6, description='cls_id'…